In [168]:
import requests
import pandas as pd
import pyspark
from pprint import pprint

if not sc:
    sc = pyspark.SparkContext()

type(sc)

pyspark.context.SparkContext

# Objectifs :

   - construire un dataframe pandas qui regroupe l'ensemble des données que sur lesquelles on veut lancer notre apprentissage
     - données statiques additionnelles à ajouter:
        1 coefficient d'isolation (nombre de stations accessible depuis la station dans un rayon donné raisonnable 5km ?)
        associer la station météo la plus proche de la station vélib
        
$e^{i\pi} + 1 = 0$

$$e^x=\sum_{i=0}^\infty \frac{1}{i!}x^i$$
       



# recupération des stations velibs de paris

In [21]:
urlParisStations = "https://api.jcdecaux.com/vls/v1/stations"
jcdKey = "c7c379f39e07e619be9663cfce805b8b266dce1a"
# https://api.jcdecaux.com/vls/v1/stations?contract={contract_name} 
stations_json = requests.get(url = urlParisStations, params={"contract":"Paris","apiKey":jcdKey}).text


In [184]:
df_stations = pd.read_json(stations_json,precise_float=True)\
                .drop("contract_name",axis=1)\
                .set_index("number")
df_stations.head()

,address,available_bike_stands,available_bikes,banking,bike_stands,bonus,last_update,name,position,status
number,,,,,,,,,,
31705,RUE DES CHAMPEAUX (PRES DE LA GARE ROUTIERE) -...,48,2,True,50,True,1489489741000,31705 - CHAMPEAUX (BAGNOLET),"{'lat': 48.8645278209514, 'lng': 2.41617072442...",OPEN
10042,52 RUE D'ENGHIEN / ANGLE RUE DU FAUBOURG POISS...,0,33,True,33,False,1489489649000,10042 - POISSONNIÈRE - ENGHIEN,"{'lat': 48.87242006305313, 'lng': 2.3483952362...",OPEN
8020,74 BOULEVARD DES BATIGNOLLES - 75008 PARIS,32,12,True,44,True,1489489725000,08020 - METRO ROME,"{'lat': 48.882148945631904, 'lng': 2.319860054...",OPEN
1022,37 RUE CASANOVA - 75001 PARIS,1,36,True,37,False,1489489512000,01022 - RUE DE LA PAIX,"{'lat': 48.8682170167744, 'lng': 2.33049351139...",OPEN
35014,139 AVENUE JEAN LOLIVE / MAIL CHARLES DE GAULL...,0,18,True,25,False,1489489725000,35014 - DE GAULLE (PANTIN),"{'lat': 48.893268664697416, 'lng': 2.412715733...",OPEN


# Récupération des stations météo de paris et alentours

source openWeather api:

http://api.openweathermap.org/data/2.5/weather?q={city name}

http://api.openweathermap.org/data/2.5/weather?q={city name},{country code}

http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}

APPID : 679875fddff53d1517c9f4acda29535e

In [54]:
first_station = df_stations.head(1).position.values[0]
first_station

{'lat': 48.8645278209514, 'lng': 2.416170724425901}

eclater la colonne postion qui contient la latitude et la longitude en 2 colonnes lat et lng

df_stations contient les coordonnées à plat

In [185]:

dfs = [df_stations.drop('position',axis=1),df_stations["position"].apply(pd.Series)]
df_stations = pd.concat(dfs,axis=1)



In [151]:
df_stations.head()

,address,available_bike_stands,available_bikes,banking,bike_stands,bonus,last_update,name,status,lat,lng
number,,,,,,,,,,,
31705,RUE DES CHAMPEAUX (PRES DE LA GARE ROUTIERE) -...,48,2,True,50,True,1489489741000,31705 - CHAMPEAUX (BAGNOLET),OPEN,48.864528,2.416171
10042,52 RUE D'ENGHIEN / ANGLE RUE DU FAUBOURG POISS...,0,33,True,33,False,1489489649000,10042 - POISSONNIÈRE - ENGHIEN,OPEN,48.872420,2.348395
8020,74 BOULEVARD DES BATIGNOLLES - 75008 PARIS,32,12,True,44,True,1489489725000,08020 - METRO ROME,OPEN,48.882149,2.319860
1022,37 RUE CASANOVA - 75001 PARIS,1,36,True,37,False,1489489512000,01022 - RUE DE LA PAIX,OPEN,48.868217,2.330494
35014,139 AVENUE JEAN LOLIVE / MAIL CHARLES DE GAULL...,0,18,True,25,False,1489489725000,35014 - DE GAULLE (PANTIN),OPEN,48.893269,2.412716


associer la station météo de chez openweathermap qui correspond à la stations vélib par geolocalisation (cityId)

http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}


APPID  : 679875fddff53d1517c9f4acda29535e



In [171]:
owmUrl = "http://api.openweathermap.org/data/2.5/weather"
appid="679875fddff53d1517c9f4acda29535e" 
#resp = requests.get(url=owmUrl,params={"APPID":appid,"lat":48.864528,"lon":2.416171})


def cityIdSerie(velib):
    return requests.get(url=owmUrl,params={"APPID":appid,"lat":velib.lat,"lon":velib.lng}).json()["id"]
    
city_ids = df_stations.apply(cityIdSerie,axis=1)



number
31705    3035403
10042    6545270
8020     3024597
1022     6545270
35014    2988621
20040    2994540
28002    2985034
15111    2970761
12124    2968054
9021     6545270
7007     6545270
20011    2994540
33006    3036386
5005     2968815
11029    2994540
4020     2978048
7012     6269531
19019    6614030
14113    2968815
15028    2970761
22005    2990611
20027    3035403
18042    6545270
11008    2994540
1028     6269531
19038    6614030
13037    3003737
13040    6457187
16027    3031137
15118    6618620
          ...   
15110    6451970
16122    2990611
7020     6545270
41605    2978621
16124    2985034
15123    6618620
5106     6269531
22001    2990611
12039    6613141
1008     6269531
31006    6613142
16037    3031137
2007     6545270
5110     2988507
12011    2978621
10151    2977824
16021    2990611
15050    2970761
20503    2994540
31709    3035403
13110    3003737
21503    2973675
1017     6545270
17006    2977824
7006     6545270
8041     2998975
21311    6451970
19121  

In [189]:

df_cities = pd.DataFrame(data=city_ids,columns=["owm_city_id"])

df_stations = pd.merge(left=df_stations,right=df_cities,how="left",left_index=True,right_index=True)



In [191]:
df_stations.head()

,address,available_bike_stands,available_bikes,banking,bike_stands,bonus,last_update,name,status,lat,lng,owm_city_id
number,,,,,,,,,,,,
31705,RUE DES CHAMPEAUX (PRES DE LA GARE ROUTIERE) -...,48,2,True,50,True,1489489741000,31705 - CHAMPEAUX (BAGNOLET),OPEN,48.864528,2.416171,3035403
10042,52 RUE D'ENGHIEN / ANGLE RUE DU FAUBOURG POISS...,0,33,True,33,False,1489489649000,10042 - POISSONNIÈRE - ENGHIEN,OPEN,48.872420,2.348395,6545270
8020,74 BOULEVARD DES BATIGNOLLES - 75008 PARIS,32,12,True,44,True,1489489725000,08020 - METRO ROME,OPEN,48.882149,2.319860,3024597
1022,37 RUE CASANOVA - 75001 PARIS,1,36,True,37,False,1489489512000,01022 - RUE DE LA PAIX,OPEN,48.868217,2.330494,6545270
35014,139 AVENUE JEAN LOLIVE / MAIL CHARLES DE GAULL...,0,18,True,25,False,1489489725000,35014 - DE GAULLE (PANTIN),OPEN,48.893269,2.412716,2988621


In [193]:
len(df_stations["owm_city_id"].unique())

54